**RAG System for text to text model Using LangChain, ChromaDB, and Groq**

In [ ]:


from google.colab import drive # Connects Google Drive to Colab
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -r requirment.txt

#Installs all required libraries (LangChain, Chroma, transformers, etc.)
"""chromadb==0.5.5
langchain==0.2.11
langchain-community==0.2.10
langchain-chroma==0.1.2
langchain-text-splitters==0.2.2
langchain-groq==0.1.6
transformers==4.43.2
sentence-transformers==3.0.1
unstructured==0.15.0
unstructured[pdf]==0.15.0"""

In [ ]:
import os  #Imports Python’s OS module to work with environment variables and file paths



from langchain.document_loaders import UnstructuredFileLoader # Loads and reads PDF files page-by-page into LangChain Document objects
from langchain.text_splitter import CharacterTextSplitter # Splits long text into smaller overlapping chunks intelligently
from langchain_community.embeddings import HuggingFaceEmbeddings # Converts text chunks into numerical vectors using a transformer model
#This enables semantic search
from langchain_chroma import Chroma # Vector database used to store and search embeddings efficiently
from langchain_groq import ChatGroq # Connects to Groq-hosted LLMs (very fast inference)
from langchain.chains import RetrievalQA # Combines retriever + LLM into a single Question-Answering pipeline



In [ ]:
os.environ["GROQ_API_KEY"] = "xyz"
# ChatGroq automatically reads this variable when initializing the LLM.

In [ ]:
# fetch pdf
import requests
url="https://pythonclassroomdiary.wordpress.com/wp-content/uploads/2019/06/functions-inpython.pdf"
response = requests.get(url) # It sends a GET request to a URL and stores the server’s response

In [ ]:
response # response stores the server’s reply after making an HTTP request, 200 successful

<Response [200]>

In [ ]:
# save pdf in local drive
# It saves the downloaded binary content into a PDF file on disk

with open("python_inbuildfunction.pdf", "wb") as f:
    f.write(response.content)

In [ ]:
!pip install -U pypdf # installs or updates the pypdf library so Python can read and work with PDF files

In [ ]:
from langchain.document_loaders import PyPDFLoader # # Import the PDF loader from LangChain

loader = PyPDFLoader("python_inbuildfunction.pdf") # # Load the PDF file named "python_inbuildfunction.pdf"
documents = loader.load() # # Read all pages of the PDF into a list of Document objects

print(len(documents)) # # Print the number of pages/documents loaded
print(documents[0].page_content[:300]) # # Print the first 300 characters of the first page
# preprocess PDFs before feeding them into NLP or search pipelines.

10
 pg. 1 www.pythonclassroomdiary.wordpress.com   by Sangeeta M Chuahan PGT CS, KV NO.3 Gwalior 
1. Functions in Python  
Function is a block of code written to carry out a specified task. Functions provide better 
modularity and a high degree of code reusing. 
 You can Pass Data(input) known as para


In [ ]:
from langchain.text_splitter import CharacterTextSplitter # # Import the text splitter from LangChain
# # Initialize a text splitter that splits text into chunks
text_splitter = CharacterTextSplitter(
    chunk_size=200, #  Each chunk will have up to 200 characters
    chunk_overlap=50 # # Consecutive chunks will overlap by 50 characters
)

# Overlap ensures context is not lost between consecutive chunks, which is useful for semantic search or LLM inpu

In [ ]:
text_splitter

In [ ]:
texts=text_splitter.split_documents(documents)
# Split the entire PDF into smaller chunks while preserving document metadata
# now have manageable chunks for NLP tasks like embeddings, semantic search and Overlap ensures the chunks retain context between splits

In [ ]:
type(texts) # texts is a Python list containing the smaller chunks of PDF

#page_content → the text of the chunk
#metadata → info like page number or source
#now have a list of text chunks, ready for further processing like embeddings, search, or LLM queries

list

In [ ]:
texts[2]

Document(metadata={'source': 'python_inbuildfunction.pdf', 'page': 2}, page_content='pg. 3 www.pythonclassroomdiary.wordpress.com   by Sangeeta M Chuahan PGT CS, KV NO.3 Gwalior \n2. Parameter (argument) Passing \nWe can define UDFs in one of the following ways in Python \n1. Function with no argument and no Return value   [ like MyMsg1(),Add() ] \n2. Function with no argument and with Return value \n3. Python Function with argument and No Return value [like MyMsg2() ] \n4. Function with argument and Return value \n \n# Python Function with No Arguments, and No Return Value  FUNCTION 1 \ndef Add1(): \n    a = 20 \n    b = 30 \n    Sum = a + b \n    print("After Calling the Function:", Sum) \nAdd1() \n \n# Python Function with Arguments, and No Return Value FUNCTION 2 \ndef Add2(a,b): \n    Sum = a + b \n        print("Result:", Sum) \nAdd2(20,30)  \n \n# Python Function with Arguments, and Return Value FUNCTION 3 \ndef Add3(a,b): \n    Sum = a + b \n    Return Sum \n \nZ=Add3(10,12) \n

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings # # Import the HuggingFace embeddings wrapper from LangChain community modules

#
# Initialize embeddings using a pre-trained sentence transformer model
# HuggingFace model helps to  generate vector representations (embeddings) for text.
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2" # # Small, fast model for sentence embeddings
)

/tmp/ipython-input-2671871813.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as se

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory = "vector_db"
# it  Define a directory where the vector database (embeddings) will be stored


In [ ]:
from langchain_chroma import Chroma
vectordb = Chroma.from_documents(texts,embeddings, persist_directory = persist_directory)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
#retrival
# Convert the vector database into a retriever object for semantic search
retriever = vectordb.as_retriever()
# vectordb is assumed to be your vector store (like Chroma) containing embeddings of your PDF chunks

In [ ]:
llm= ChatGroq(model = "llama-3.3-70b-versatile",temperature=0)  # Controls randomness; 0 = deterministic responses
# # Initialize a ChatGroq LLM (large language model) for generating responses
# The model to use (LLAMA 70B variant)


In [ ]:
# Create a RetrievalQA chain that combines LLM reasoning with semantic search

qa_chain = RetrievalQA.from_chain_type( # RetrievalQA is a LangChain chain designed for question answering over documents
    llm=llm, chain_type = "stuff" , #  The LLM to generate answers
    # Chain type: "stuff" all retrieved docs are concatenated as context
    retriever=retriever,  # Semantic retriever to fetch relevant chunks
    return_source_documents=True # return the documents used to generate the answer
)

#chain_type="stuff": concatenates all retrieved chunks into a single prompt for the LLM. Other options like "map_reduce" or "refine" handle context differently.

#retriever: provides the relevant chunks from your vector store for each query.

#return_source_documents=True: gives the exact chunks that contributed to the answer, useful for verification or citations

In [ ]:
query = "what are the inbuild function from this pdf"

response= qa_chain({"query": query})

#qa_chain is a RetrievalQA object combining the LLM and the retriever


/tmp/ipython-input-2529433063.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  response= qa_chain({"query": query})
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
print(response) # LLM-generated answer

{'query': 'what are the inbuild function from this pdf', 'result': 'According to the provided context, the following are some of the built-in functions in Python:\n\n1. abs()\n2. divmod()\n3. input()\n4. open()\n5. staticmethod()\n6. all()\n7. enumerate()\n8. int()\n9. ord()\n10. str()\n11. any()\n12. eval()\n13. isinstance()\n14. pow()\n15. sum()\n16. basestring()\n17. execfile()\n18. issubclass()\n19. print()\n20. super()\n21. bin()\n22. file()\n23. iter()\n24. property()\n25. tuple()\n26. bool()\n27. filter()\n28. len()\n29. range()\n30. type()\n31. bytearray()\n32. float()\n33. list()\n34. raw_input()\n35. unichr()\n36. callable()\n37. format()\n38. locals()\n39. reduce()\n40. unicode()\n41. chr()\n42. frozenset()\n43. long()\n44. reload()\n45. vars()\n46. classmethod()\n47. getattr()\n48. map()\n49. repr()\n50. xrange()\n51. compile()\n52. globals()\n53. max()\n54. reversed()\n55. zip()\n56. complex()\n57. hash()\n58. memoryview()\n59. round()\n60. __import__()\n61. delattr()\n62.

In [ ]:
print(response["result"])

According to the provided context, the following are some of the built-in functions in Python:

1. abs()
2. divmod()
3. input()
4. open()
5. staticmethod()
6. all()
7. enumerate()
8. int()
9. ord()
10. str()
11. any()
12. eval()
13. isinstance()
14. pow()
15. sum()
16. basestring()
17. execfile()
18. issubclass()
19. print()
20. super()
21. bin()
22. file()
23. iter()
24. property()
25. tuple()
26. bool()
27. filter()
28. len()
29. range()
30. type()
31. bytearray()
32. float()
33. list()
34. raw_input()
35. unichr()
36. callable()
37. format()
38. locals()
39. reduce()
40. unicode()
41. chr()
42. frozenset()
43. long()
44. reload()
45. vars()
46. classmethod()
47. getattr()
48. map()
49. repr()
50. xrange()
51. compile()
52. globals()
53. max()
54. reversed()
55. zip()
56. complex()
57. hash()
58. memoryview()
59. round()
60. __import__()
61. delattr()
62. help()
63. next()
64. setattr()
65. dict()
66. hex()
67. object()
68. slice()
69. dir()
70. id()
71. oct()
72. sorted()

Additiona

In [ ]:
query = "Give me a summary of all functions from this PDF."

response = qa_chain.invoke({"query": query}) #  Run the RetrievalQA chain to get an answer

print(response["result"]) # Print the LLM-generated answer
print("*" * 30) # Print a separator for readability

#PDF → split into chunks

#Chunks → embeddings → vector store

#Retriever → find relevant chunks

#LLM → generate human-readable summary

The provided text lists the following functions:

**Built-in Functions:**

1. abs()
2. all()
3. any()
4. basestring()
5. bin()
6. bool()
7. bytearray()
8. callable()
9. chr()
10. classmethod()
11. cmp()
12. compile()
13. complex()
14. delattr()
15. dict()
16. dir()
17. divmod()
18. enumerate()
19. eval()
20. execfile()
21. file()
22. filter()
23. float()
24. format()
25. frozenset()
26. getattr()
27. globals()
28. hasattr()
29. hash()
30. help()
31. hex()
32. id()
33. input()
34. int()
35. isinstance()
36. issubclass()
37. iter()
38. len()
39. list()
40. locals()
41. long()
42. map()
43. max()
44. memoryview()
45. min()
46. next()
47. object()
48. oct()
49. open()
50. ord()
51. pow()
52. print()
53. property()
54. range()
55. raw_input()
56. reduce()
57. reload()
58. repr()
59. reversed()
60. round()
61. set()
62. setattr()
63. slice()
64. sorted()
65. staticmethod()
66. str()
67. sum()
68. super()
69. tuple()
70. type()
71. unichr()
72. unicode()
73. vars()
74. xrange()
75. zip()
76. 

In [ ]:
query = "Give me a summery on the topic isinstance()."

response = qa_chain.invoke({"query": query})

print(response["result"])
print("*" * 30)


The `isinstance()` function in Python is a built-in function that checks if an object (first argument) is an instance or subclass of a class (second argument). It returns `True` if the object is an instance or subclass of the class, and `False` otherwise.

The general syntax of `isinstance()` is:
```
isinstance(object, class)
```
Where `object` is the object to be checked, and `class` is the class to check against.

For example:
```
x = 5
print(isinstance(x, int))  # Returns: True
print(isinstance(x, str))   # Returns: False
```
In this example, `x` is an integer, so `isinstance(x, int)` returns `True`, while `isinstance(x, str)` returns `False` because `x` is not a string.

`isinstance()` can also be used to check if an object is an instance of a subclass of a particular class. For example:
```
class Animal:
    pass

class Dog(Animal):
    pass

my_dog = Dog()
print(isinstance(my_dog, Dog))    # Returns: True
print(isinstance(my_dog, Animal)) # Returns: True
```
In this example, `my_

In [ ]:
# Imports
"""chromadb==0.5.5
langchain==0.2.11
langchain-community==0.2.10
langchain-chroma==0.1.2
langchain-text-splitters==0.2.2
langchain-groq==0.1.6
transformers==4.43.2
sentence-transformers==3.0.1
unstructured==0.15.0
unstructured[pdf]==0.15.0"""

'chromadb==0.5.5\nlangchain==0.2.11\nlangchain-community==0.2.10\nlangchain-chroma==0.1.2\nlangchain-text-splitters==0.2.2\nlangchain-groq==0.1.6\ntransformers==4.43.2\nsentence-transformers==3.0.1\nunstructured==0.15.0\nunstructured[pdf]==0.15.0'

**Conclusion**

This project implements a Retrieval-Augmented Generation (RAG) system that allows users to ask natural language questions and receive accurate, context-aware answers extracted from a PDF document.

Instead of relying only on a language model’s memory, the system retrieves relevant document content from a vector database and augments the LLM response with grounded information.

RAG system flow:

PDF Document ↓ Document Loading ↓ Text Chunking ↓ Vector Embeddings ↓ ChromaDB (Vector Store) ↓ Retriever ↓ Groq LLM (LLaMA 3.3) ↓ Accurate Text Response

-> Environment Setup & Dependencies -Google Colab is used as the execution environment. -Required libraries are installed using a requirements.txt file. -The Groq API key is set as an environment variable to securely access Groq’s LLM services.

The PDF is saved locally for further processing
-> Document Loading (PDF Parsing) -PyPDFLoader from LangChain is used to: -Read the PDF -Convert each page into a structured document object

-> Text Chunking (Preprocessing) -CharacterTextSplitter splits the document into , Chunk size: 200 characters , Overlap: 50 characters

overlap ensures Context is not lost between chunks
-> Embedding Generation (Semantic Representation)

HuggingFaceEmbeddings using:Each text chunk is converted into a numerical vector
These embeddings capture semantic meaning
->Vector Storage with ChromaDB -Vectors are persisted locally (persist_directory="vector_db") -Persistent storage for reuse,Easy LangChain integration using chromaDB

-> Retriever Creation -ChromaDB is converted into a retriever using , vectordb.as_retriever() -Finds the most relevant text chunks, Uses vector similarity instead of keyword matching

-> LLM Integration Using Groq API -llama-3.3-70b-versatile, Temperature is set to 0 for deterministic and factual responses

->RetrievalQA Chain (RAG Core) -RetrievalQA chain connects:Retriever (ChromaDB), LLM (Groq)

these steps involved: -User submits a query -Relevant document chunks are retrieved -Retrieved context is injected into the LLM prompt -LLM generates a grounded answer

->Finally This project demonstrates a production-ready RAG system that overcomes traditional LLM limitations by grounding responses in external documents. By combining LangChain orchestration, ChromaDB vector storage, and Groq’s high-performance LLMs, the system delivers accurate, explainable, and context-aware answers from unstructured data sources like PDFs.